# Regression Models in R (tips)

In [7]:
if(!exists("Table1", mode="function")) source("mechkar.R")

In [8]:

library(readr)
library(dplyr)
library(ggplot2)


In [9]:
df <- read.csv("train.csv")
head(df)
dim(df)

,id,season,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,1,1,1,0,6,0,2,0.344167,0.363625,0.805833,0.1604460,985
2,2,1,1,0,0,0,2,0.363478,0.353739,0.696087,0.2485390,801
3,3,1,1,0,1,1,1,0.196364,0.189405,0.437273,0.2483090,1349
4,4,1,1,0,2,1,1,0.200000,0.212122,0.590435,0.1602960,1562
5,5,1,1,0,3,1,1,0.226957,0.229270,0.436957,0.1869000,1600
6,6,1,1,0,4,1,1,0.204348,0.233209,0.518261,0.0895652,1606


[1] 365  12

Data Set Information:

Bike sharing systems are new generation of traditional bike rentals where whole process from membership, rental and return back has become automatic. Through these systems, user is able to easily rent a bike from a particular position and return back at another position. Currently, there are about over 500 bike-sharing programs around the world which is composed of over 500 thousands bicycles. Today, there exists great interest in these systems due to their important role in traffic, environmental and health issues.

Apart from interesting real world applications of bike sharing systems, the characteristics of data being generated by these systems make them attractive for the research. Opposed to other transport services such as bus or subway, the duration of travel, departure and arrival position is explicitly recorded in these systems. This feature turns bike sharing system into a virtual sensor network that can be used for sensing mobility in the city. Hence, it is expected that most of important events in the city could be detected via monitoring these data.


Attribute Information:

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv

- instant: record index
- season : season (1:winter, 2:spring, 3:summer, 4:fall)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from [Web Link])
- weekday : day of the week
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
+ weathersit :
- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-8, t_max=+39 (only in hourly scale)
- atemp: Normalized feeling temperature in Celsius. The values are derived via (t-t_min)/(t_max-t_min), t_min=-16, t_max=+50 (only in hourly scale)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered

# EDA

In [10]:
summary(df)

       id          season           mnth           holiday      
 Min.   :  1   Min.   :1.000   Min.   : 1.000   Min.   :0.0000  
 1st Qu.: 92   1st Qu.:2.000   1st Qu.: 4.000   1st Qu.:0.0000  
 Median :183   Median :3.000   Median : 7.000   Median :0.0000  
 Mean   :183   Mean   :2.499   Mean   : 6.526   Mean   :0.0274  
 3rd Qu.:274   3rd Qu.:3.000   3rd Qu.:10.000   3rd Qu.:0.0000  
 Max.   :365   Max.   :4.000   Max.   :12.000   Max.   :1.0000  
    weekday        workingday       weathersit         temp        
 Min.   :0.000   Min.   :0.0000   Min.   :1.000   Min.   :0.05913  
 1st Qu.:1.000   1st Qu.:0.0000   1st Qu.:1.000   1st Qu.:0.32500  
 Median :3.000   Median :1.0000   Median :1.000   Median :0.47917  
 Mean   :3.008   Mean   :0.6849   Mean   :1.422   Mean   :0.48666  
 3rd Qu.:5.000   3rd Qu.:1.0000   3rd Qu.:2.000   3rd Qu.:0.65667  
 Max.   :6.000   Max.   :1.0000   Max.   :3.000   Max.   :0.84917  
     atemp              hum           windspeed            cnt      


In [11]:
df$holiday <- as.factor(df$holiday)
df$season <- as.factor(df$season)
df$mnth <- as.factor(df$mnth)
df$workingday <- as.factor(df$workingday)
df$weathersit <- as.factor(df$weathersit)
df$weekday <- as.factor(df$weekday)
summary(df)

       id      season      mnth     holiday weekday workingday weathersit
 Min.   :  1   1:90   1      : 31   0:355   0:52    0:115      1:226     
 1st Qu.: 92   2:92   3      : 31   1: 10   1:52    1:250      2:124     
 Median :183   3:94   5      : 31           2:52               3: 15     
 Mean   :183   4:89   7      : 31           3:52                         
 3rd Qu.:274          8      : 31           4:52                         
 Max.   :365          10     : 31           5:52                         
                      (Other):179           6:53                         
      temp             atemp              hum           windspeed      
 Min.   :0.05913   Min.   :0.07907   Min.   :0.0000   Min.   :0.02239  
 1st Qu.:0.32500   1st Qu.:0.32195   1st Qu.:0.5383   1st Qu.:0.13558  
 Median :0.47917   Median :0.47285   Median :0.6475   Median :0.18690  
 Mean   :0.48666   Mean   :0.46684   Mean   :0.6437   Mean   :0.19140  
 3rd Qu.:0.65667   3rd Qu.:0.61238   3rd Qu.:0.7

In [6]:
exploreData(data=df)

  |================================================================      |  92%

# DATASET PARTITION

In [12]:
tab1 <- train_test(data=df, train_name="train", test_name="test", prop=0.7, seed=5, tableone=TRUE)
tab1

Dataset partitioned into:

 + Train dataset: train

 + Test dataset: test

Warning message in Table1(data = df, y = "traintest_ind_", messages = FALSE):
"The following variables have unique values and will not be included in the analysis: "


  |======================================================================| 100%

Warning message:
"The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
Using compatibility `.name_repair`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated."


  |===========================                                           |  38%

Warning message in stats::chisq.test(data[[v]], data[[y]]):
"Chi-squared approximation may be incorrect"


  |=================================================================     |  92%

 

You got a perfectly balanced training and test datasets

 



V1,V2,Pop,1,2,pval
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Individuals,n,365,255,110,NA
id,Mean (SD),183.0 (105.5),183.7 (105.0),181.4 (107.2),NA
id,Median (IQR),183.0 (92.0-274.0),181.0 (92.5-272.5),184.5 (88.0-277.0),0.854
season,1,90 (24.7%),62 (24.3%),28 (25.5%),NA
season,2,92 (25.2%),67 (26.3%),25 (22.7%),0.914
season,3,94 (25.8%),65 (25.5%),29 (26.4%),NA
season,4,89 (24.4%),61 (23.9%),28 (25.5%),NA
mnth,1,31 (8.5%),20 (7.8%),11 (10.0%),NA
mnth,2,28 (7.7%),18 (7.1%),10 (9.1%),0.841


# MODELS

In [13]:
### The error we will use is the RMSE and RMSLE
rmse <- function(y,y_hat) {
    err <- sqrt(sum((y_hat-y)^2,na.rm=T)/length(y))
    return(err)
}

rmsle <- function(y,y_hat) {
    err <- sqrt(sum((log(y_hat+1)-log(y+1))^2,na.rm=T)/length(y))
    return(err)
}


In [14]:
### Table of resulting errors
### Name, Model, RMSE, RMSLE
err_res <- NULL

## Linear Models

In [10]:
## model with only the original variables
mod1 <- lm(cnt~., data=train)
summary(mod1)



Call:
lm(formula = cnt ~ ., data = train)

Residuals:
    Min      1Q  Median      3Q     Max 
-2351.2  -281.8    49.3   315.1  1643.8 

Coefficients: (1 not defined because of singularities)
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1765.418    304.169   5.804 2.17e-08 ***
id             -5.444      4.133  -1.317 0.189147    
season2       480.129    212.698   2.257 0.024943 *  
season3       868.711    252.799   3.436 0.000702 ***
season4      1231.007    215.068   5.724 3.29e-08 ***
mnth2         390.491    226.880   1.721 0.086595 .  
mnth3         663.199    309.957   2.140 0.033455 *  
mnth4        1290.897    464.783   2.777 0.005939 ** 
mnth5        2251.882    568.274   3.963 9.94e-05 ***
mnth6        2106.798    690.979   3.049 0.002570 ** 
mnth7        1347.358    826.740   1.630 0.104553    
mnth8        1851.927    927.585   1.997 0.047079 *  
mnth9        2199.177   1028.174   2.139 0.033515 *  
mnth10       2396.705   1151.136   2.082 0.038465 *

In [11]:
pred1 <- predict(mod1,newdata=test)
rmse(test$cnt,pred1)
rmsle(test$cnt,pred1)


Warning message in predict.lm(mod1, newdata = test):
"prediction from a rank-deficient fit may be misleading"


[1] 633.0224

[1] 0.2467678

In [12]:
err_res <- rbind(err_res, data.frame(Name="Base Linear regression", Model="mod1", 
                                     RMSE=rmse(test$cnt,pred1), 
                                     RMSLE=rmsle(test$cnt,pred1)))

In [13]:
err_res

Name,Model,RMSE,RMSLE
<chr>,<chr>,<dbl>,<dbl>
Base Linear regression,mod1,641.8646,0.2402681


## Desicion trees

In [14]:
library(tree)
library(rpart)

In [15]:
mod3 <- tree(cnt ~., data=train)
mod3

node), split, n, deviance, yval
      * denotes terminal node

 1) root 255 487700000 3406  
   2) id < 107 77  35270000 1807  
     4) id < 69.5 48  10940000 1485 *
     5) id > 69.5 29  11160000 2339 *
   3) id > 107 178 170500000 4097  
     6) atemp < 0.457063 54  39100000 3203  
      12) weathersit: 3 5   4075000 1660 *
      13) weathersit: 1,2 49  21900000 3361  
        26) id < 354.5 43  12410000 3498 *
        27) id > 354.5 6   2844000 2375 *
     7) atemp > 0.457063 124  69400000 4487  
      14) hum < 0.8475 112  34100000 4609 *
      15) hum > 0.8475 12  18110000 3349 *

In [16]:
pred3 <- predict(mod3,newdata=test)
rmse(test$cnt,pred3)
rmsle(test$cnt,pred3)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-tree", Model="mod3", 
                                     RMSE=rmse(test$cnt,pred3), 
                                     RMSLE=rmsle(test$cnt,pred3)))

[1] 620.6197

[1] 0.2648109

In [18]:
mod4 <- rpart(cnt ~., data=train)
mod4

n= 255 

node), split, n, deviance, yval
      * denotes terminal node

 1) root 255 491193900 3421.408  
   2) temp< 0.45875 116 123091300 2264.129  
     4) id< 106.5 69  25473170 1681.261  
       8) id< 36.5 28   4350789 1262.107 *
       9) id>=36.5 41  12843530 1967.512 *
     5) id>=106.5 47  39761840 3119.830  
      10) hum>=0.828958 7   5489309 1870.714 *
      11) hum< 0.828958 40  21439150 3338.425  
        22) season=1,2 9   4446874 2597.444 *
        23) season=4 31  10616190 3553.548 *
   3) temp>=0.45875 139  83093220 4387.194  
     6) id< 109 8   1370632 3059.500 *
     7) id>=109 131  66759210 4468.275  
      14) hum>=0.7560415 29  22147810 3876.345 *
      15) hum< 0.7560415 102  31561420 4636.569 *

In [19]:
pred4 <- predict(mod4,newdata=test)
rmse(test$cnt,pred4)
rmsle(test$cnt,pred4)
err_res <- rbind(err_res, data.frame(Name="Decision Trees-rpart", Model="mod4", 
                                     RMSE=rmse(test$cnt,pred4), 
                                     RMSLE=rmsle(test$cnt,pred4)))

[1] 653.9241

[1] 0.258213

ERROR: Error in rbind(err_res, data.frame(Name = "Decision Trees-rpart", Model = "mod4", : object 'err_res' not found


## Random Forest

In [20]:
library(randomForest)
library(ranger)

randomForest 4.6-14

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'randomForest'


The following object is masked from 'package:ggplot2':

    margin


The following object is masked from 'package:dplyr':

    combine



Attaching package: 'ranger'


The following object is masked from 'package:randomForest':

    importance




In [ ]:
#mod5 <- randomForest(tip ~., data=train)
#mod5

In [ ]:
#pred5 <- predict(mod5,newdata=test)
#rmse(test$tip,pred5)
#rmsle(test$tip,pred5)

In [ ]:
mod6 <- ranger(tip ~., data=train)
mod6

In [ ]:
pred6 <- predict(mod6,data=test)
#head(pred6)
rmse(test$tip,pred6$predictions)
rmsle(test$tip,pred6$predictions)
err_res <- rbind(err_res, data.frame(Name="RandomForest (ranger)", Model="mod6", 
                                     RMSE=rmse(test$tip,pred6$predictions), 
                                     RMSLE=rmsle(test$tip,pred6$predictions)))

## XGBoost

In [15]:
library(xgboost)

Warning message:
"package 'xgboost' was built under R version 4.0.3"

Attaching package: 'xgboost'


The following object is masked from 'package:dplyr':

    slice




In [16]:
train1 <- Matrix::sparse.model.matrix(tip ~ .-1, data = train)

ERROR: Error in eval(predvars, data, env): object 'tip' not found


In [17]:
test1 <- Matrix::sparse.model.matrix(tip ~ .-1, data = test)

ERROR: Error in eval(predvars, data, env): object 'tip' not found


In [ ]:
#X_train <- xgb.DMatrix(train1)
X_train <- train1
y_train <- train$tip
mod7 <- xgboost(data=X_train,label=y_train, nrounds=100,print_every_n = 10)

In [ ]:
#X_test <- xgb.DMatrix(test1)
X_test <- test1
y_test <- test$tip

pred7 <- predict(mod7,newdata=X_test)
rmse(y_test,pred7)
rmsle(y_test,pred7)
err_res <- rbind(err_res, data.frame(Name="XGBoost", Model="mod7", 
                                     RMSE=rmse(test$tip,pred7), 
                                     RMSLE=rmsle(test$tip,pred7)))

## kNN 

In [ ]:
### adaboost needs that values to be normalized
min_max <- function(x) { (x -min(x))/(max(x)-min(x))   }

In [ ]:
X_train <- sapply(data.frame(as.matrix(train1)),min_max)

In [ ]:
X_test <- sapply(data.frame(as.matrix(test1)),min_max)

In [ ]:
summary(X_train)

In [ ]:
library(class)
mod8 <- knn(X_train,X_test,cl=train$tip)

In [ ]:
str(mod8)

In [ ]:
pred8 <- as.numeric(as.character(mod8))

rmse(test$tip,pred8)
rmsle(test$tip,pred8)
err_res <- rbind(err_res, data.frame(Name="kNN", Model="mod8", 
                                     RMSE=rmse(test$tip,pred8), 
                                     RMSLE=rmsle(test$tip,pred8)))

## SVM

In [ ]:
#install.packages("liquidSVM")
library(liquidSVM)

mod9 <- svm(tip ~., train)

In [ ]:
pred9 <- predict(mod9, newdata=test)

rmse(test$tip,pred9)
rmsle(test$tip,pred9)
err_res <- rbind(err_res, data.frame(Name="SVM", Model="mod9", 
                                     RMSE=rmse(test$tip,pred9), 
                                     RMSLE=rmsle(test$tip,pred9)))

In [ ]:
err_res %>% arrange(RMSLE)